In [1]:
import numpy as np
import cv2
import dlib
import matplotlib.pyplot as plt
import matplotlib.image as img

In [2]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

RuntimeError: Unable to open shape_predictor_68_face_landmarks.dat

In [ ]:
# 얼굴 감지와 특징점 추정
def face(img):
    
    landmarks = []
    option = 0
    
    #dlib 사용
    pro_face, scores, idx = detector.run(img)
    dl_cnt = len(pro_face)
    score = 0
    
    if dl_cnt == 1:
        for f in pro_face:
            land = predictor(img, f)
            face_img = img[f.top():f.bottom(), f.left():f.right()].copy()
            score = scores[0]

            for l in land.parts():
                landmarks.append([l.x, l.y])
        option = 1
    elif dl_cnt > 1:
        num = 0
        for i, sco in enumerate(scores):
            if sco > score:
                score = sco
                num = i
        for i, f in enumerate(pro_face):
            if i == num:
                land = predictor(img, f)
                face_img = img[f.top():f.bottom(), f.left():f.right()].copy()

                for l in land.parts():
                    landmarks.append([l.x, l.y])
        option = 1
    else:
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        filtered_img = cv2.GaussianBlur(gray, (121, 121), 15)

        [faces, neighbours, weights] = faceCascade.detectMultiScale3(filtered_img, scaleFactor=1.05, minNeighbors=6 ,flags=cv2.CASCADE_SCALE_IMAGE , minSize=(100,100), outputRejectLevels = True)
        ha_cnt = len(faces)
        
        if ha_cnt == 1:
            for (x, y, w, h) in faces:
                dlib_rect = dlib.rectangle(int(x), int(y), int(x+w), int(y+h))
                face_img = img[int(y):int(y+h), int(x):int(x+w)].copy()
                
                land = predictor(img, dlib_rect)
                for l in land.parts():
                    landmarks.append([l.x, l.y])

                # landmarks = np.matrix([[p.x, p.y] for p in predictor(gray, dlib_rect).parts()])
            score = weights[0]
            option = 2
        elif ha_cnt > 1:
            num = 0
            for i, sco in enumerate(weights):
                if sco > score:
                    score = sco
                    num = i
            for i, (x, y, w, h) in enumerate(pro_face):
                if i == num:
                    dlib_rect = dlib.rectangle(int(x), int(y), int(x+w), int(y+h))
                    face_img = img[int(y):int(y+h), int(x):int(x+w)].copy()

                    landmarks = np.matrix([[p.x, p.y] for p in predictor(gray, dlib_rect).parts()])
            option = 2       
        else:
            face_img = img.copy()
            score = -1
    
    return landmarks, face_img

In [ ]:
# 얼굴의 중심 좌표를 출력
def center(landmarks):
    sum_x = [0] * 3
    sum_y = [0] * 3
    x = [0] * 3
    y = [0] * 3
    center_point = [0] * 2
    for i in range(36, 68):
        if i <= 41:
            sum_x[0] += landmarks[i][0]
            sum_y[0] += landmarks[i][1]
        elif i <= 47:
            sum_x[1] += landmarks[i][0]
            sum_y[1] += landmarks[i][1]
        else:
            sum_x[2] += landmarks[i][0]
            sum_y[2] += landmarks[i][1]
    for i in range(0, 3):
        if i == 2:
            x[i] = sum_x[i] // 20
            y[i] = sum_y[i] // 20
        else:
            x[i] = sum_x[i] // 6
            y[i] = sum_y[i] // 6
    
    center_point[0] = sum(x) // 3
    center_point[1] = sum(y) // 3

    return center_point

In [ ]:
# center2가 center1이 되도록 img를 이동
def center_trans(center1, center2, img):
    height, width, _ = img.shape
    T = np.float32([[1,0,(center1[0] - center2[0])],[0,1,(center1[1] - center2[1])]])
    trans = cv2.warpAffine(img, T, (width, height), cv2.INTER_LINEAR)
    
    return trans

In [ ]:
# 입력
front_img = cv2.imread('front_1.jpg')
profile_img = cv2.imread('profile_1.jpg')
before_img = profile_img.copy()

# 얼굴 감지, 특징점 추정
fro_landmarks, fro_img = face(front_img)
pro_landmarks, pro_img = face(profile_img)

# 얼굴의 중심 맞추기
fro_center = center(fro_landmarks)
pro_center = center(pro_landmarks)
profile_img = center_trans(fro_center, pro_center, profile_img)
pro_landmarks, pro_img = face(profile_img)

pro_points = np.array(pro_landmarks)
fro_points = np.array(fro_landmarks)

# Homography 변환 행렬구하기
H, _ = cv2.findHomography(pro_points, fro_points, cv2.RANSAC)

# 변환
result = cv2.warpPerspective(profile_img, H, (profile_img.shape[1], profile_img.shape[0]))

In [ ]:
fig = plt.figure(figsize = (20, 35))
rows = 1
cols = 2

ax1 = fig.add_subplot(rows, cols, 1)
ax1.imshow(cv2.cvtColor(before_img, cv2.COLOR_BGR2RGB))
ax1.set_title('Before')
ax1.axis("off")


ax2 = fig.add_subplot(rows, cols, 2)
ax2.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
ax2.set_title('After')
ax2.axis("off")